In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_squared_error
from scipy.special import expit  # Sigmoid function for propensity score
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
from sklearn.neural_network import MLPClassifier, MLPRegressor

In [ ]:
from pathlib import Path
import os
import glob
from joblib import dump, load
import pandas as pd
import scipy
import scipy.stats
import scipy.special
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from grr.utils.riesznet import RieszNet
from grr.utils.moments import ate_moment_fn
from grr.utils.ihdp_data import *


In [ ]:
moment_fn = ate_moment_fn

In [ ]:
drop_prob = 0.0  # dropout prob of dropout layers throughout notebook
n_hidden = 100  # width of hidden layers throughout notebook

# Training params
learner_lr = 1e-5
learner_l2 = 1e-3
learner_l1 = 0.0
n_epochs = 600
earlystop_rounds = 40 # how many epochs to wait for an out-of-sample improvement
earlystop_delta = 1e-4
target_reg = 1.0
riesz_weight = 0.1

bs = 64
device = torch.cuda.current_device() if torch.cuda.is_available() else None
print("GPU:", torch.cuda.is_available())

from itertools import chain, combinations
from itertools import combinations_with_replacement as combinations_w_r

def _combinations(n_features, degree, interaction_only):
        comb = (combinations if interaction_only else combinations_w_r)
        return chain.from_iterable(comb(range(n_features), i)
                                   for i in range(0, degree + 1))

class Learner(nn.Module):

    def __init__(self, n_t, n_hidden, p, degree, interaction_only=False):
        super().__init__()
        n_common = 200
        self.monomials = list(_combinations(n_t, degree, interaction_only))
        self.common = nn.Sequential(nn.Dropout(p=p), nn.Linear(n_t, n_common), nn.ELU(),
                                    nn.Dropout(p=p), nn.Linear(n_common, n_common), nn.ELU(),
                                    nn.Dropout(p=p), nn.Linear(n_common, n_common), nn.ELU())
        self.riesz_nn = nn.Sequential(nn.Dropout(p=p), nn.Linear(n_common, 1))
        self.riesz_poly = nn.Sequential(nn.Linear(len(self.monomials), 1))
        self.reg_nn0 = nn.Sequential(nn.Dropout(p=p), nn.Linear(n_common, n_hidden), nn.ELU(),
                                    nn.Dropout(p=p), nn.Linear(n_hidden, n_hidden), nn.ELU(),
                                    nn.Dropout(p=p), nn.Linear(n_hidden, 1))
        self.reg_nn1 = nn.Sequential(nn.Dropout(p=p), nn.Linear(n_common, n_hidden), nn.ELU(),
                                    nn.Dropout(p=p), nn.Linear(n_hidden, n_hidden), nn.ELU(),
                                    nn.Dropout(p=p), nn.Linear(n_hidden, 1))
        self.reg_poly = nn.Sequential(nn.Linear(len(self.monomials), 1))


    def forward(self, x):
        poly = torch.cat([torch.prod(x[:, t], dim=1, keepdim=True)
                          for t in self.monomials], dim=1)
        feats = self.common(x)
        riesz = self.riesz_nn(feats) + self.riesz_poly(poly)
        reg = self.reg_nn0(feats) * (1 - x[:, [0]]) + self.reg_nn1(feats) * x[:, [0]] + self.reg_poly(poly)
        return torch.cat([reg, riesz], dim=1)

In [ ]:
drop_prob = 0.0  # dropout prob of dropout layers throughout notebook
n_hidden = 100  # width of hidden layers throughout notebook

# Training params
learner_lr = 1e-5
learner_l2 = 1e-3
learner_l1 = 0.0
n_epochs = 600
earlystop_rounds = 40 # how many epochs to wait for an out-of-sample improvement
earlystop_delta = 1e-4
target_reg = 1.0
riesz_weight = 0.1

bs = 64
device = torch.cuda.current_device() if torch.cuda.is_available() else None
print("GPU:", torch.cuda.is_available())

from itertools import chain, combinations
from itertools import combinations_with_replacement as combinations_w_r

def _combinations(n_features, degree, interaction_only):
        comb = (combinations if interaction_only else combinations_w_r)
        return chain.from_iterable(comb(range(n_features), i)
                                   for i in range(0, degree + 1))

class Learner(nn.Module):

    def __init__(self, n_t, n_hidden, p, degree, interaction_only=False):
        super().__init__()
        n_common = 200
        self.monomials = list(_combinations(n_t, degree, interaction_only))
        self.common = nn.Sequential(nn.Dropout(p=p), nn.Linear(n_t, n_common), nn.ELU(),
                                    nn.Dropout(p=p), nn.Linear(n_common, n_common), nn.ELU(),
                                    nn.Dropout(p=p), nn.Linear(n_common, n_common), nn.ELU())
        self.riesz_nn = nn.Sequential(nn.Dropout(p=p), nn.Linear(n_common, 1))
        self.riesz_poly = nn.Sequential(nn.Linear(len(self.monomials), 1))
        self.reg_nn0 = nn.Sequential(nn.Dropout(p=p), nn.Linear(n_common, n_hidden), nn.ELU(),
                                    nn.Dropout(p=p), nn.Linear(n_hidden, n_hidden), nn.ELU(),
                                    nn.Dropout(p=p), nn.Linear(n_hidden, 1))
        self.reg_nn1 = nn.Sequential(nn.Dropout(p=p), nn.Linear(n_common, n_hidden), nn.ELU(),
                                    nn.Dropout(p=p), nn.Linear(n_hidden, n_hidden), nn.ELU(),
                                    nn.Dropout(p=p), nn.Linear(n_hidden, 1))
        self.reg_poly = nn.Sequential(nn.Linear(len(self.monomials), 1))


    def forward(self, x):
        poly = torch.cat([torch.prod(x[:, t], dim=1, keepdim=True)
                          for t in self.monomials], dim=1)
        feats = self.common(x)
        riesz = self.riesz_nn(feats) + self.riesz_poly(poly)
        reg = self.reg_nn0(feats) * (1 - x[:, [0]]) + self.reg_nn1(feats) * x[:, [0]] + self.reg_poly(poly)
        return torch.cat([reg, riesz], dim=1)

In [ ]:
# Simulation parameters
n = 3000 # Number of samples
p = 3   # Number of covariates
treatment_effect = 5.0  # True treatment effect

# Generate covariates
np.random.seed(0)

In [ ]:
true_ATE = treatment_effect

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

class NeuralNetBiasCorrection:
    def __init__(self, input_dim, hidden_dim=100, max_iter=3000, tol=1e-10, lbd=0.01, loss="Logit", lr=0.01):
        """
        Bias-correction model using a neural network
        :param input_dim: Dimension of input features
        :param hidden_dim: Number of hidden units in the hidden layer
        :param max_iter: Maximum number of iterations (epochs)
        :param tol: Convergence tolerance
        :param lbd: Weight of the regularization term
        :param loss: Loss function type (e.g., "Logit", "DBCLS", "DBCLogit")
        :param lr: Learning rate
        """
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.max_iter = max_iter
        self.tol = tol
        self.lbd = lbd
        self.lr = lr
        
        # Build the neural network
        self.model = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ELU(),
            nn.Linear(hidden_dim, 1),
            nn.Sigmoid()
        )
        
        if loss == "Logit":
            self.criterion = self._logistic_loss_function
        elif loss == "CBPS":
            self.criterion = self._cbps_loss_function
        elif loss == "DBCLS":
            self.criterion = self._least_squares_loss_function
        elif loss == "DBCUKL":
            self.criterion = self._ukl_loss_function
        elif loss == "DBCEB":
            self.criterion = self._eb_loss_function
        else:
            raise ValueError("Invalid loss function specified")

        self.optimizer = optim.Adam(self.model.parameters(), lr=self.lr)
        
    
    def _least_squares_loss_function(self, X_tensor, outputs, targets):
        """Least-squares loss function."""
        outputs = torch.clamp(outputs, min=0.05, max=0.95)
        loss = torch.mean(-2 * (1 / outputs + 1 / (1 - outputs)) + (targets / outputs - (1 - targets) / (1 - outputs))**2)
        return loss + self.lbd * self._l2_regularization()
    
    def _ukl_loss_function(self, X_tensor, outputs, targets):
        """Constrained logistic-style loss function."""
        outputs = torch.clamp(outputs, min=0.05, max=0.95)
        # Correcting the formula with the proper variable names
        loss = - torch.log(1/outputs) - torch.log(1/(1 - outputs)) + targets / outputs + (1 - targets) / (1 - outputs)
        # Mean loss calculation
        loss = torch.mean(loss)
        return loss + self.lbd * self._l2_regularization()
    
    def _eb_loss_function(self, X_tensor, outputs, targets):
        """Constrained logistic-style loss function."""
        outputs = torch.clamp(outputs, min=0.05, max=0.95)
        # Correcting the formula with the proper variable names
        loss = - (1 - targets)*torch.log(1/outputs - 1) - targets*torch.log(1/(1 - outputs) - 1) + targets*(1/outputs) + (1 - targets)*(1/(1 - outputs))
        # Mean loss calculation
        loss = torch.mean(loss)
        return loss + self.lbd * self._l2_regularization()
    
    def _logistic_loss_function(self, X_tensor, outputs, targets):
        """Logistic loss function."""
        outputs = torch.clamp(outputs, min=0.05, max=0.95)
        loss = nn.BCELoss()(outputs, targets)  # Binary cross-entropy
        return loss + self.lbd * self._l2_regularization()
    
    def _cbps_loss_function(self, X_tensor, outputs, targets):
        """Logistic loss function."""
        outputs = torch.clamp(outputs, min=0.05, max=0.95)
        loss =  torch.mean((targets * X_tensor / outputs - (1 - targets) * X_tensor / (1 - outputs))**2, axis=0)
        loss = (loss**2).mean()
        return loss + self.lbd * self._l2_regularization()

    def _l2_regularization(self):
        """Compute the L2 regularization term."""
        reg_loss = sum(torch.sum(param**2) for param in self.model.parameters())
        return reg_loss
    
    def fit(self, X, T, batch_size=100):
        """
        Train the model (mini-batch training)
        :param X: Covariates (array of shape N×d)
        :param T: Targets (binary array of shape N×1)
        :param batch_size: Mini-batch size
        """
        X_tensor = torch.tensor(X, dtype=torch.float32)
        T_tensor = torch.tensor(T, dtype=torch.float32).view(-1, 1)

        dataset = TensorDataset(X_tensor, T_tensor)
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

        prev_loss = float('inf')
        for epoch in range(self.max_iter):
            for X_batch, T_batch in dataloader:
                self.optimizer.zero_grad()
                outputs = self.model(X_batch)
                loss = self.criterion(X_batch, outputs, T_batch)  # Use the mini-batch inputs (X_batch) when evaluating the loss
                loss.backward()
                self.optimizer.step()

            # Convergence check (per epoch)
            current_loss = loss.item()
            if abs(prev_loss - current_loss) < self.tol:
                break
            prev_loss = current_loss

    def predict_proba(self, X):
        """
        Compute predicted probabilities
        :param X: Covariates (array of shape N×d)
        :return: Probability of class 1
        """
        X_tensor = torch.tensor(X, dtype=torch.float32)
        with torch.no_grad():
            probas = torch.clamp(self.model(X_tensor), min=0.05, max=0.95).numpy()
        return np.hstack([1 - probas, probas])

    def predict(self, X):
        """
        Predict class labels
        :param X: Covariates (array of shape N×d)
        :return: Predicted class (0 or 1)
        """
        probas = self.predict_proba(X)
        return (probas[:, 1] >= 0.5).astype(int)

    def get_params(self):
        """
        Get trained parameters
        :return: Model weights and biases
        """
        params = {name: param.detach().numpy() for name, param in self.model.named_parameters()}
        return params


### Data generation

In [ ]:
num_trial = 300

In [ ]:
nsims = 1000
np.random.seed(123)
#sim_ids = np.random.choice(len(simulation_files), nsims, replace=False)
methods = ['dr', 'direct', 'ips']
srr = {'dr' : True, 'direct' : False, 'ips' : True}


In [ ]:
result_list = []

for tr in range(num_trial):
    print(tr)
    result_list_temp = []
    
    X = np.random.normal(0, 1, (n, p))

    # Define a propensity score model
    # Assume treatment probability is a sigmoid function of a subset of covariates
    X_temp = np.concatenate([X, X**2, np.array([X[:, 0]*X[:, 1], X[:, 1]*X[:, 2], X[:, 0]*X[:, 2]]).T], axis=1)
    propensity_coef = np.random.normal(0, 0.5, X_temp.shape[1])
    propensity_scores = expit(X_temp @ propensity_coef)  # Calculate propensity scores

    # Generate treatment assignment based on propensity scores
    T = np.random.binomial(1, propensity_scores)

    # Generate outcome with treatment effect
    # Assume a simple linear model for demonstration
    beta = np.random.normal(0, 1, p)
    Y = (X @ beta)**2 + 1.1 + treatment_effect * T + np.random.normal(0, 1, n)

    X_treatment = X[T == 1]
    X_control = X[T == 0]
    
    y_scaler = StandardScaler(with_mean=True).fit(np.array([Y]).T)
    y = y_scaler.transform(np.array([Y]).T)
    XT = np.c_[T, X]
    
    X_train, X_test, y_train, y_test = train_test_split(XT, y, test_size = 0.2)
                            
    Y = y.T[0]
    
    Y_treatment = Y[T == 1]
    Y_control = Y[T == 0]
    
    XT = np.c_[T, X]
    
    outcome_model = MLPRegressor(random_state=1, max_iter=600)
    outcome_model.fit(XT, Y)

    T_treatment = T*0 + 1
    T_control = T*0
    XT_treatment = np.c_[T_treatment, X]
    XT_control = np.c_[T_control, X]

    est_treatment_outcome = outcome_model.predict(XT_treatment)
    est_control_outcome = outcome_model.predict(XT_control)
    
    for method in ["DBCLS", "DBCUKL", "DBCEB", "Logit"]:
        prop_model = NeuralNetBiasCorrection(input_dim=p, lbd = 0., loss=method)
        prop_model.fit(X, T)
        est_prop_score = prop_model.predict_proba(X)[:, 1]
        est_prop_score_dbc = est_prop_score

        #treatment_outcome_model = KernelRegression(kernel="rbf", gamma=np.logspace(-2, 2, 10))
        #control_outcome_model = KernelRegression(kernel="rbf", gamma=np.logspace(-2, 2, 10))

        IPW_est = np.mean(T*Y / est_prop_score - (1 - T)*Y / (1 - est_prop_score))

        # Evaluate performance
        IPW_bias = IPW_est - true_ATE

        result_list_temp.append(IPW_est)

        DM_est = np.mean(est_treatment_outcome - est_control_outcome)

        # Evaluate performance
        DM_bias = DM_est - true_ATE
        
        result_list_temp.append(DM_est)

        DR_est = np.mean(T*(Y - est_treatment_outcome) / est_prop_score - (1 - T)*(Y - est_control_outcome)  / (1 - est_prop_score) + est_treatment_outcome - est_control_outcome)

        # Evaluate performance
        DR_bias = DR_est - true_ATE

        result_list_temp.append(DR_est)

    result_list.append(result_list_temp)

In [ ]:
np.savetxt("result", result_list)

In [ ]:
np.mean(np.array(result_list)**2, axis=0)

In [ ]:
np.array(result_list).shape

In [ ]:
res = tuple(np.array(x) for x in zip(*result_list2))
truth = res[-1:]
res_dict = {}

res_list_temp = []
for it, method in enumerate(methods):
    point, lb, ub = res[it * 3: (it + 1)*3]
    res_list_temp.append(point)

In [ ]:
result_list_final = np.concatenate([np.array(result_list), np.array(res_list_temp).T], axis=1)

In [ ]:
np.savetxt("ATE_result1.csv", result_list_final)

In [ ]:
np.round(np.sqrt(np.mean((result_list_final - true_ATE)**2, axis=0)), 3)

In [ ]:
np.mean((np.array(res_list_temp).T - true_ATE)**2, axis=0)

In [ ]:
np.array(result_list)

In [ ]:
res = tuple(np.array(x) for x in zip(*result_list2))
truth = res[-1:]
res_dict = {}

res_list_temp = []
for it, method in enumerate(methods):
    point, lb, ub = res[it * 3: (it + 1)*3]
    res_list_temp.append(point)

In [ ]:
np.concatenate([np.array(result_list), np.array(res_list_temp).T], axis=1)

In [ ]:
np.array(res_list_temp).T.shape

In [ ]:
res_dict

In [ ]:
np.array(result_list).shape

In [ ]:
result_list = []

for tr in range(num_trial):
    result_list_temp = []
    
    
    X = np.random.normal(0, 1, (n, p))

    # Define a propensity score model
    # Assume treatment probability is a sigmoid function of a subset of covariates
    X_temp = np.concatenate([X, X**2, np.array([X[:, 0]*X[:, 1], X[:, 1]*X[:, 2], X[:, 0]*X[:, 2]]).T], axis=1)
    propensity_coef = np.random.normal(0, 0.5, X_temp.shape[1])
    propensity_scores = expit(X_temp @ propensity_coef)  # Calculate propensity scores

    # Generate treatment assignment based on propensity scores
    T = np.random.binomial(1, propensity_scores)

    # Generate outcome with treatment effect
    # Assume a simple linear model for demonstration
    beta = np.random.normal(0, 1, p)
    Y = (X @ beta)**2 + 1.1 + treatment_effect * T + np.random.normal(0, 1, n)

    X_treatment = X[T == 1]
    X_control = X[T == 0]

    Y_treatment = Y[T == 1]
    Y_control = Y[T == 0]
    
    #### Direct bias correction
    prop_model = NeuralNetBiasCorrection(input_dim=p, lbd = 0.01, loss="DBCLS")
    prop_model.fit(X, T)
    est_prop_score = prop_model.predict_proba(X)[:, 1]
    est_prop_score_dbc = est_prop_score

    treatment_outcome_model = KernelRegression(kernel="rbf", gamma=np.logspace(-2, 2, 10))
    control_outcome_model = KernelRegression(kernel="rbf", gamma=np.logspace(-2, 2, 10))

    treatment_outcome_model.fit(X_treatment, Y_treatment)
    control_outcome_model.fit(X_control, Y_control)

    est_treatment_outcome = treatment_outcome_model.predict(X)
    est_control_outcome = control_outcome_model.predict(X)

    IPW_est = np.mean(T*Y / est_prop_score - (1 - T)*Y / (1 - est_prop_score))

    # Evaluate performance
    IPW_bias = IPW_est - true_ATE

    print(f"Estimated ATE: {IPW_est}")
    print(f"Bias: {IPW_bias}")
    
    result_list_temp.append(IPW_est)
    
    DM_est = np.mean(est_treatment_outcome - est_control_outcome)

    # Evaluate performance
    DM_bias = DM_est - true_ATE

    print(f"Estimated ATE: {DM_est}")
    print(f"Bias: {DM_bias}")
    
    result_list_temp.append(DM_est)

    DR_est = np.mean(T*(Y - est_treatment_outcome) / est_prop_score - (1 - T)*(Y - est_control_outcome)  / (1 - est_prop_score) + est_treatment_outcome - est_control_outcome)

    # Evaluate performance
    DR_bias = DR_est - true_ATE

    print(f"Estimated ATE: {DR_est}")
    print(f"Bias: {DR_bias}")
    
    result_list_temp.append(DR_est)
    
    #### Direct bias correction
    prop_model = NeuralNetBiasCorrection(input_dim=p, lbd = 0.01, loss="CBPS")
    prop_model.fit(X, T)
    est_prop_score = prop_model.predict_proba(X)[:, 1]
    est_prop_score_dbc = est_prop_score

    treatment_outcome_model = KernelRegression(kernel="rbf", gamma=np.logspace(-2, 2, 10))
    control_outcome_model = KernelRegression(kernel="rbf", gamma=np.logspace(-2, 2, 10))

    treatment_outcome_model.fit(X_treatment, Y_treatment)
    control_outcome_model.fit(X_control, Y_control)

    est_treatment_outcome = treatment_outcome_model.predict(X)
    est_control_outcome = control_outcome_model.predict(X)

    IPW_est = np.mean(T*Y / est_prop_score - (1 - T)*Y / (1 - est_prop_score))

    # Evaluate performance
    IPW_bias = IPW_est - true_ATE

    print(f"Estimated ATE: {IPW_est}")
    print(f"Bias: {IPW_bias}")
    
    result_list_temp.append(IPW_est)
    
    DM_est = np.mean(est_treatment_outcome - est_control_outcome)

    # Evaluate performance
    DM_bias = DM_est - true_ATE

    print(f"Estimated ATE: {DM_est}")
    print(f"Bias: {DM_bias}")
    
    result_list_temp.append(DM_est)

    DR_est = np.mean(T*(Y - est_treatment_outcome) / est_prop_score - (1 - T)*(Y - est_control_outcome)  / (1 - est_prop_score) + est_treatment_outcome - est_control_outcome)

    # Evaluate performance
    DR_bias = DR_est - true_ATE

    print(f"Estimated ATE: {DR_est}")
    print(f"Bias: {DR_bias}")
    
    result_list_temp.append(DR_est)
    
    ##### Linear models
    
    # Fit a linear model to estimate the treatment effect
    model = LinearRegression()
    model.fit(np.hstack([X, T.reshape(-1, 1)]), Y)
    estimated_treatment_effect = model.coef_[-1]

    # Evaluate performance
    true_ATE = treatment_effect
    bias = estimated_treatment_effect - true_ATE
    mse = mean_squared_error(Y, model.predict(np.hstack([X, T.reshape(-1, 1)])))

    print(f"Estimated ATE: {estimated_treatment_effect}")
    print(f"Bias: {bias}")
    print(f"Mean Squared Error: {mse}")

    result_list_temp.append(estimated_treatment_effect)
    
    #### Logistc regression 
    
    prop_model = NeuralNetBiasCorrection(input_dim=p, lbd = 0., loss="Logit")
    prop_model.fit(X, T)
    est_prop_score = prop_model.predict_proba(X)[:, 1]

    treatment_outcome_model = KernelRegression(kernel="rbf", gamma=np.logspace(-2, 2, 10))
    control_outcome_model = KernelRegression(kernel="rbf", gamma=np.logspace(-2, 2, 10))

    treatment_outcome_model.fit(X_treatment, Y_treatment)
    control_outcome_model.fit(X_control, Y_control)

    est_treatment_outcome = treatment_outcome_model.predict(X)
    est_control_outcome = control_outcome_model.predict(X)

    IPW_est = np.mean(T*Y / est_prop_score - (1 - T)*Y / (1 - est_prop_score))

    # Evaluate performance
    IPW_bias = IPW_est - true_ATE

    print(f"Estimated ATE: {IPW_est}")
    print(f"Bias: {IPW_bias}")
    
    result_list_temp.append(IPW_est)
    
    DM_est = np.mean(est_treatment_outcome - est_control_outcome)

    # Evaluate performance
    DM_bias = DM_est - true_ATE

    print(f"Estimated ATE: {DM_est}")
    print(f"Bias: {DM_bias}")
    
    result_list_temp.append(DM_est)

    DR_est = np.mean(T*(Y - est_treatment_outcome) / est_prop_score - (1 - T)*(Y - est_control_outcome)  / (1 - est_prop_score) + est_treatment_outcome - est_control_outcome)

    # Evaluate performance
    DR_bias = DR_est - true_ATE

    print(f"Estimated ATE: {DR_est}")
    print(f"Bias: {DR_bias}")
    
    result_list_temp.append(DR_est)
    
    #### CBPS
    
    # Enable automatic conversion of Pandas DataFrame to R DataFrame
    pandas2ri.activate()

    # Simulate data in Python

    # Create a pandas DataFrame
    column_names = [f'X{i+1}' for i in range(p)]
    df = pd.DataFrame(X, columns=column_names)
    df['T'] = T
    df['Y'] = Y


    # Convert pandas DataFrame to R DataFrame
    r_df = pandas2ri.py2rpy(df)

    ro.r.assign("p", p)

    # Load the CBPS package in R and fit the model for ATE estimation
    ro.r('''
        library(CBPS)
        estimate_cbps_ate <- function(df) {
            formula_str <- paste("T ~", paste(names(df)[1:{p}], collapse=" + "))

            # Apply CBPS (estimate the ATE, ATT=0)
            model <- CBPS(as.formula(formula_str), data = df, ATT = 0, method = "exact")

            # Retrieve the estimated propensity scores
            df$propensity_score <- fitted(model)

            # Apply IPW (Inverse Probability Weighting)
            df$weight <- ifelse(df$T == 1, 1 / df$propensity_score, 1 / (1 - df$propensity_score))

            # Estimate the ATE via weighted regression
            result <- lm(Y ~ T, data = df, weights = df$weight)

            return(df$propensity_score)
        }
    ''')

    # Call the R function to obtain propensity scores (and the ATE)
    est_prop_score = ro.r['estimate_cbps_ate'](r_df)
    
    est_prop_score_cbps = est_prop_score
    
    #print(er)

    treatment_outcome_model = KernelRegression(kernel="rbf", gamma=np.logspace(-2, 2, 10))
    control_outcome_model = KernelRegression(kernel="rbf", gamma=np.logspace(-2, 2, 10))

    treatment_outcome_model.fit(X_treatment, Y_treatment)
    control_outcome_model.fit(X_control, Y_control)

    est_treatment_outcome = treatment_outcome_model.predict(X)
    est_control_outcome = control_outcome_model.predict(X)

    IPW_est = np.mean(T*Y / est_prop_score - (1 - T)*Y / (1 - est_prop_score))

    # Evaluate performance
    IPW_bias = IPW_est - true_ATE

    print(f"Estimated ATE: {IPW_est}")
    print(f"Bias: {IPW_bias}")
    
    result_list_temp.append(IPW_est)
    
    DM_est = np.mean(est_treatment_outcome - est_control_outcome)

    # Evaluate performance
    DM_bias = DM_est - true_ATE

    print(f"Estimated ATE: {DM_est}")
    print(f"Bias: {DM_bias}")
    
    result_list_temp.append(DM_est)

    DR_est = np.mean(T*(Y - est_treatment_outcome) / est_prop_score - (1 - T)*(Y - est_control_outcome)  / (1 - est_prop_score) + est_treatment_outcome - est_control_outcome)

    # Evaluate performance
    DR_bias = DR_est - true_ATE

    print(f"Estimated ATE: {DR_est}")
    print(f"Bias: {DR_bias}")
    
    result_list_temp.append(DR_est)
    
    
    result_list.append(result_list_temp)


In [ ]:
np.round(np.sqrt(np.mean((np.array(result_list) - true_ATE)**2, axis=0)), 3)

In [ ]:
res = tuple(np.array(x) for x in zip(*result_list2))
truth = res[-1:]
res_dict = {}
for it, method in enumerate(methods):
    point, lb, ub = res[it * 3: (it + 1)*3]
    res_dict[method] = {'point': point, 'lb': lb, 'ub': ub,
                        'cov': np.mean(np.logical_and(truth >= lb, truth <= ub)),
                        'bias': np.mean(point - truth),
                        'rmse': rmse_fn(point, truth)
                        }
    print("{} : bias = {:.3f}, rmse = {:.3f}, cov = {:.3f}".format(method, res_dict[method]['bias'], res_dict[method]['rmse'], res_dict[method]['cov']))

In [ ]:
np.round(np.sqrt(np.mean((np.array(result_list) - true_ATE)**2, axis=0)), 3)

In [ ]:
np.concatenate([X, X**2], axis=1).shape

In [ ]:
X_temp

In [ ]:
np.max(propensity_scores)

In [ ]:
np.min(propensity_scores)

In [ ]:
np.max(est_prop_score_dbc)

In [ ]:
np.min(est_prop_score_dbc)

In [ ]:
np.max(est_prop_score_cbps)

In [ ]:
np.min(est_prop_score_cbps)

In [ ]:
np.round(np.sqrt(np.mean((np.array(result_list) - true_ATE)**2, axis=0)), 3)

In [ ]:
print(f"True Average Treatment Effect (ATE): {true_ATE}")

In [ ]:
# Fit a linear model to estimate the treatment effect
model = LinearRegression()
model.fit(np.hstack([X, T.reshape(-1, 1)]), Y)
estimated_treatment_effect = model.coef_[-1]

# Evaluate performance
true_ATE = treatment_effect
bias = estimated_treatment_effect - true_ATE
mse = mean_squared_error(Y, model.predict(np.hstack([X, T.reshape(-1, 1)])))

print(f"Estimated ATE: {estimated_treatment_effect}")
print(f"Bias: {bias}")
print(f"Mean Squared Error: {mse}")





In [ ]:
# Fit a linear model to estimate the treatment effect
prop_model = LogisticRegression()
prop_model.fit(X, T)
est_prop_score = prop_model.predict_proba(X)[:, 1]

treatment_outcome_model = KernelRegression(kernel="rbf", gamma=np.logspace(-2, 2, 10))
control_outcome_model = KernelRegression(kernel="rbf", gamma=np.logspace(-2, 2, 10))

treatment_outcome_model.fit(X_treatment, Y_treatment)
control_outcome_model.fit(X_control, Y_control)

est_treatment_outcome = treatment_outcome_model.predict(X)
est_control_outcome = control_outcome_model.predict(X)

IPW_est = np.mean(T*Y / est_prop_score - (1 - T)*Y / (1 - est_prop_score))

# Evaluate performance
IPW_bias = IPW_est - true_ATE

print(f"Estimated ATE: {IPW_est}")
print(f"Bias: {IPW_bias}")

DR_est = np.mean(T*(Y - est_treatment_outcome) / est_prop_score - (1 - T)*(Y - est_control_outcome)  / (1 - est_prop_score) + est_treatment_outcome - est_control_outcome)

# Evaluate performance
DR_bias = DR_est - true_ATE

print(f"Estimated ATE: {DR_est}")
print(f"Bias: {DR_bias}")

DM_est = np.mean(est_treatment_outcome - est_control_outcome)

# Evaluate performance
DM_bias = DM_est - true_ATE

print(f"Estimated ATE: {DM_est}")
print(f"Bias: {DM_bias}")

In [ ]:
                                                              result_list = []

for tr in range(num_trial):
    result_list_temp = []
    
    
    X = np.random.normal(0, 1, (n, p))

    # Define a propensity score model
    # Assume treatment probability is a sigmoid function of a subset of covariates
    propensity_coef = np.random.normal(0, 0.5, p)
    propensity_scores = expit(X @ propensity_coef)  # Calculate propensity scores

    # Generate treatment assignment based on propensity scores
    T = np.random.binomial(1, propensity_scores)

    # Generate outcome with treatment effect
    # Assume a simple linear model for demonstration
    beta = np.random.normal(0, 1, p)
    Y = (X @ beta)**2 + 1.1 + treatment_effect * T + np.random.normal(0, 1, n)

    X_treatment = X[T == 1]
    X_control = X[T == 0]

    Y_treatment = Y[T == 1]
    Y_control = Y[T == 0]
    
        #### Direct bias correction
    prop_model = DirectBiasCorrection(lbd = 0.01)
    
    prop_model.fit(X, T, Y)
    est_prop_score = prop_model.predict_proba(X)[:, 1]
    est_prop_score_dbc = est_prop_score

    treatment_outcome_model = KernelRegression(kernel="rbf", gamma=np.logspace(-2, 2, 10))
    control_outcome_model = KernelRegression(kernel="rbf", gamma=np.logspace(-2, 2, 10))

    treatment_outcome_model.fit(X_treatment, Y_treatment)
    control_outcome_model.fit(X_control, Y_control)

    est_treatment_outcome = treatment_outcome_model.predict(X)
    est_control_outcome = control_outcome_model.predict(X)

    IPW_est = np.mean(T*Y / est_prop_score - (1 - T)*Y / (1 - est_prop_score))

    # Evaluate performance
    IPW_bias = IPW_est - true_ATE

    print(f"Estimated ATE: {IPW_est}")
    print(f"Bias: {IPW_bias}")
    
    result_list_temp.append(IPW_est)
    
    DM_est = np.mean(est_treatment_outcome - est_control_outcome)

    # Evaluate performance
    DM_bias = DM_est - true_ATE

    print(f"Estimated ATE: {DM_est}")
    print(f"Bias: {DM_bias}")
    
    result_list_temp.append(DM_est)

    DR_est = np.mean(T*(Y - est_treatment_outcome) / est_prop_score - (1 - T)*(Y - est_control_outcome)  / (1 - est_prop_score) + est_treatment_outcome - est_control_outcome)

    # Evaluate performance
    DR_bias = DR_est - true_ATE

    print(f"Estimated ATE: {DR_est}")
    print(f"Bias: {DR_bias}")
    
    result_list_temp.append(DR_est)
    
    ##### Linear models
    
    # Fit a linear model to estimate the treatment effect
    model = LinearRegression()
    model.fit(np.hstack([X, T.reshape(-1, 1)]), Y)
    estimated_treatment_effect = model.coef_[-1]

    # Evaluate performance
    true_ATE = treatment_effect
    bias = estimated_treatment_effect - true_ATE
    mse = mean_squared_error(Y, model.predict(np.hstack([X, T.reshape(-1, 1)])))

    print(f"Estimated ATE: {estimated_treatment_effect}")
    print(f"Bias: {bias}")
    print(f"Mean Squared Error: {mse}")

    result_list_temp.append(estimated_treatment_effect)
    
    
    
    #### Logistc regression 
    
    prop_model = LogisticRegression()
    prop_model.fit(X, T)
    est_prop_score = prop_model.predict_proba(X)[:, 1]

    treatment_outcome_model = KernelRegression(kernel="rbf", gamma=np.logspace(-2, 2, 10))
    control_outcome_model = KernelRegression(kernel="rbf", gamma=np.logspace(-2, 2, 10))

    treatment_outcome_model.fit(X_treatment, Y_treatment)
    control_outcome_model.fit(X_control, Y_control)

    est_treatment_outcome = treatment_outcome_model.predict(X)
    est_control_outcome = control_outcome_model.predict(X)

    IPW_est = np.mean(T*Y / est_prop_score - (1 - T)*Y / (1 - est_prop_score))

    # Evaluate performance
    IPW_bias = IPW_est - true_ATE

    print(f"Estimated ATE: {IPW_est}")
    print(f"Bias: {IPW_bias}")
    
    result_list_temp.append(IPW_est)
    
    DM_est = np.mean(est_treatment_outcome - est_control_outcome)

    # Evaluate performance
    DM_bias = DM_est - true_ATE

    print(f"Estimated ATE: {DM_est}")
    print(f"Bias: {DM_bias}")
    
    result_list_temp.append(DM_est)

    DR_est = np.mean(T*(Y - est_treatment_outcome) / est_prop_score - (1 - T)*(Y - est_control_outcome)  / (1 - est_prop_score) + est_treatment_outcome - est_control_outcome)

    # Evaluate performance
    DR_bias = DR_est - true_ATE

    print(f"Estimated ATE: {DR_est}")
    print(f"Bias: {DR_bias}")
    
    result_list_temp.append(DR_est)
    

    
    
    #### CBPS
    
    # Enable automatic conversion of Pandas DataFrame to R DataFrame
    pandas2ri.activate()

    # Simulate data in Python

    # Create a pandas DataFrame
    column_names = [f'X{i+1}' for i in range(p)]
    df = pd.DataFrame(X, columns=column_names)
    df['T'] = T
    df['Y'] = Y


    # Convert pandas DataFrame to R DataFrame
    r_df = pandas2ri.py2rpy(df)

    ro.r.assign("p", p)

    # Load the CBPS package in R and fit the model for ATE estimation
    ro.r('''
        library(CBPS)
        estimate_cbps_ate <- function(df) {
            formula_str <- paste("T ~", paste(names(df)[1:{p}], collapse=" + "))

            # Apply CBPS (estimate the ATE, ATT=0)
            model <- CBPS(as.formula(formula_str), data = df, ATT = 0, method = "exact")

            # Retrieve the estimated propensity scores
            df$propensity_score <- fitted(model)

            # Apply IPW (Inverse Probability Weighting)
            df$weight <- ifelse(df$T == 1, 1 / df$propensity_score, 1 / (1 - df$propensity_score))

            # Estimate the ATE via weighted regression
            result <- lm(Y ~ T, data = df, weights = df$weight)

            return(df$propensity_score)
        }
    ''')

    # Call the R function to obtain propensity scores (and the ATE)
    est_prop_score = ro.r['estimate_cbps_ate'](r_df)
    
    est_prop_score_cbps = est_prop_score
    
    #print(er)

    treatment_outcome_model = KernelRegression(kernel="rbf", gamma=np.logspace(-2, 2, 10))
    control_outcome_model = KernelRegression(kernel="rbf", gamma=np.logspace(-2, 2, 10))

    treatment_outcome_model.fit(X_treatment, Y_treatment)
    control_outcome_model.fit(X_control, Y_control)

    est_treatment_outcome = treatment_outcome_model.predict(X)
    est_control_outcome = control_outcome_model.predict(X)

    IPW_est = np.mean(T*Y / est_prop_score - (1 - T)*Y / (1 - est_prop_score))

    # Evaluate performance
    IPW_bias = IPW_est - true_ATE

    print(f"Estimated ATE: {IPW_est}")
    print(f"Bias: {IPW_bias}")
    
    result_list_temp.append(IPW_est)
    
    DM_est = np.mean(est_treatment_outcome - est_control_outcome)

    # Evaluate performance
    DM_bias = DM_est - true_ATE

    print(f"Estimated ATE: {DM_est}")
    print(f"Bias: {DM_bias}")
    
    result_list_temp.append(DM_est)

    DR_est = np.mean(T*(Y - est_treatment_outcome) / est_prop_score - (1 - T)*(Y - est_control_outcome)  / (1 - est_prop_score) + est_treatment_outcome - est_control_outcome)

    # Evaluate performance
    DR_bias = DR_est - true_ATE

    print(f"Estimated ATE: {DR_est}")
    print(f"Bias: {DR_bias}")
    
    result_list_temp.append(DR_est)
    
    
    result_list.append(result_list_temp)


In [ ]:
# Fit a linear model to estimate the treatment effect
prop_model = DirectBiasCorrection()
prop_model.fit(X, T)
est_prop_score = prop_model.predict_proba(X)[:, 1]

treatment_outcome_model = KernelRegression(kernel="rbf", gamma=np.logspace(-2, 2, 10))
control_outcome_model = KernelRegression(kernel="rbf", gamma=np.logspace(-2, 2, 10))

treatment_outcome_model.fit(X_treatment, Y_treatment)
control_outcome_model.fit(X_control, Y_control)

est_treatment_outcome = treatment_outcome_model.predict(X)
est_control_outcome = control_outcome_model.predict(X)

IPW_est = np.mean(T*Y / est_prop_score - (1 - T)*Y / (1 - est_prop_score))

# Evaluate performance
IPW_bias = IPW_est - true_ATE

print(f"Estimated ATE: {IPW_est}")
print(f"Bias: {IPW_bias}")

DR_est = np.mean(T*(Y - est_treatment_outcome) / est_prop_score - (1 - T)*(Y - est_control_outcome)  / (1 - est_prop_score) + est_treatment_outcome - est_control_outcome)

# Evaluate performance
DR_bias = DR_est - true_ATE

print(f"Estimated ATE: {DR_est}")
print(f"Bias: {DR_bias}")

DM_est = np.mean(est_treatment_outcome - est_control_outcome)

# Evaluate performance
DM_bias = DM_est - true_ATE

print(f"Estimated ATE: {DM_est}")
print(f"Bias: {DM_bias}")

In [ ]:
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
import pandas as pd
import numpy as np

# Enable automatic conversion of Pandas DataFrame to R DataFrame

# Simulate data in Python

# Create a pandas DataFrame
column_names = [f'X{i+1}' for i in range(p)]
df = pd.DataFrame(X, columns=column_names)
df['T'] = T
df['Y'] = Y


# Convert pandas DataFrame to R DataFrame
r_df = pandas2ri.py2rpy(df)

ro.r.assign("p", p)

# Load the CBPS package in R and fit the model for ATE estimation
ro.r('''
    library(CBPS)
    estimate_cbps_ate <- function(df) {
        formula_str <- paste("T ~", paste(names(df)[1:{p}], collapse=" + "))
        
        # Apply CBPS (estimate the ATE, ATT=0)
        model <- CBPS(as.formula(formula_str), data = df, ATT = 0, method = "exact")
        
        # Retrieve the estimated propensity scores
        df$propensity_score <- fitted(model)
        
        # Apply IPW (Inverse Probability Weighting)
        df$weight <- ifelse(df$T == 1, 1 / df$propensity_score, 1 / (1 - df$propensity_score))
        
        # Estimate the ATE via weighted regression
        result <- lm(Y ~ T, data = df, weights = df$weight)
        
        return(df$propensity_score)
    }
''')

# Call the R function to obtain propensity scores (and the ATE)
est_prop_score = ro.r['estimate_cbps_ate'](r_df)

treatment_outcome_model = KernelRegression(kernel="rbf", gamma=np.logspace(-2, 2, 10))
control_outcome_model = KernelRegression(kernel="rbf", gamma=np.logspace(-2, 2, 10))

treatment_outcome_model.fit(X_treatment, Y_treatment)
control_outcome_model.fit(X_control, Y_control)

est_treatment_outcome = treatment_outcome_model.predict(X)
est_control_outcome = control_outcome_model.predict(X)

IPW_est = np.mean(T*Y / est_prop_score - (1 - T)*Y / (1 - est_prop_score))

# Evaluate performance
IPW_bias = IPW_est - true_ATE

print(f"Estimated ATE: {IPW_est}")
print(f"Bias: {IPW_bias}")

DR_est = np.mean(T*(Y - est_treatment_outcome) / est_prop_score - (1 - T)*(Y - est_control_outcome)  / (1 - est_prop_score) + est_treatment_outcome - est_control_outcome)

# Evaluate performance
DR_bias = DR_est - true_ATE

print(f"Estimated ATE: {DR_est}")
print(f"Bias: {DR_bias}")

DM_est = np.mean(est_treatment_outcome - est_control_outcome)

# Evaluate performance
DM_bias = DM_est - true_ATE

print(f"Estimated ATE: {DM_est}")
print(f"Bias: {DM_bias}")


In [ ]:
est_prop_score

In [ ]:
DM_est = np.mean(est_treatment_outcome - est_control_outcome)

# Evaluate performance
DM_bias = DM_est - true_ATE

print(f"Estimated ATE: {DM_est}")
print(f"Bias: {DM_bias}")

In [ ]:


IPW_est = np.mean(T*Y / est_prop_score - (1 - T)*Y / (1 - est_prop_score))

# Evaluate performance
IPW_bias = IPW_est - true_ATE

print(f"Estimated ATE: {IPW_est}")
print(f"Bias: {IPW_bias}")

In [ ]:
T

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
# Convert data to PyTorch tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
T_tensor = torch.tensor(T, dtype=torch.float32).view(-1, 1)

# Define a simple neural network model for propensity score estimation
class PropensityScoreNN(nn.Module):
    def __init__(self, input_dim):
        super(PropensityScoreNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        return x

# Initialize model, loss function, and optimizer
model = PropensityScoreNN(p)
criterion = nn.BCELoss()  # Binary Cross Entropy for binary classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the neural network
num_epochs = 500
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    
    # Forward pass
    outputs = model(X_tensor)
    loss = criterion(outputs, T_tensor)
    
    # Backward pass and optimization
    loss.backward()
    optimizer.step()
    
    # Print loss for every 100 epochs
    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Estimated propensity scores for all data
with torch.no_grad():
    estimated_propensity_scores = model(X_tensor).numpy()

In [ ]:
estimated_propensity_scores[estimated_propensity_scores < 0.01] = 0.01

In [ ]:
estimated_propensity_scores[estimated_propensity_scores > 0.99] = 0.99

In [ ]:
np.mean((T/estimated_propensity_scores.T[0] - (1-T)/(1 - estimated_propensity_scores.T[0]))*Y)

In [ ]:
# Convert data to PyTorch tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
T_tensor = torch.tensor(T, dtype=torch.float32).view(-1, 1)

# Define a simple neural network model for propensity score estimation
class PropensityScoreNN(nn.Module):
    def __init__(self, input_dim):
        super(PropensityScoreNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        return x

# Initialize model, loss function, and optimizer
model = PropensityScoreNN(p)
criterion = nn.BCELoss()  # Binary Cross Entropy for binary classification
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Train the neural network
num_epochs = 500
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    
    # Forward pass
    outputs = model(X_tensor)
    outputs = torch.clamp(outputs, min=0.01, max=0.99)
    loss = -2*(1/outputs + 1/(1 - outputs)) + (T_tensor / outputs - (1-T_tensor) / (1-outputs))**2
    loss = loss.mean()
    
    # Backward pass and optimization
    loss.backward()
    optimizer.step()
    
    # Print loss for every 100 epochs
    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Estimated propensity scores for all data
with torch.no_grad():
    estimated_propensity_scores = model(X_tensor).numpy()

In [ ]:
Z_tensor = torch.cat([T_tensor, X_tensor], axis=1)
Y_tensor = torch.tensor(Y, dtype=torch.float32).view(-1, 1)
dim = Z_tensor.shape[1]

class CodOutcomeNN(nn.Module):
    def __init__(self, input_dim):
        super(CodOutcomeNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.fc2 = nn.Linear(128, 32)
        self.fc3 = nn.Linear(32, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    
# Initialize model, loss function, and optimizer
model = CodOutcomeNN(dim)
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Train the neural network
num_epochs = 10000
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    
    # Forward pass
    outputs = model(Z_tensor)
    loss = ((Y_tensor - outputs)**2).mean()
    loss = loss.mean()
    
    # Backward pass and optimization
    loss.backward()
    optimizer.step()
    
    # Print loss for every 100 epochs
    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Estimated propensity scores for all data
with torch.no_grad():
    estimated_conditional_outcomes = model(Z_tensor).numpy()
    estimated_conditional_outcomes_1 = model(Z_tensor_1).numpy()
    estimated_conditional_outcomes_0 = model(Z_tensor_0).numpy()

In [ ]:
with torch.no_grad():
    estimated_conditional_outcomes = model(Z_tensor).numpy()
    estimated_conditional_outcomes_1 = model(Z_tensor_1).numpy()
    estimated_conditional_outcomes_0 = model(Z_tensor_0).numpy()

In [ ]:
aaa = np.mean(estimated_conditional_outcomes_1 - estimated_conditional_outcomes_0)

In [ ]:
aaa

In [ ]:
aaa + np.mean((T/estimated_propensity_scores.T[0] - (1-T)/(1 - estimated_propensity_scores.T[0]))*(Y - estimated_conditional_outcomes.T[0]))

In [ ]:
Y - estimated_conditional_outcomes.T[0]